In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
import openpyxl as opx
from tqdm import tqdm

In [11]:
org2  = pd.read_csv('Org2_filtr_by_income.csv')
org2.drop_duplicates(subset=['inn'])
df = org2[['inn', 'income', 'short_name', 'full_name', 'region', 'city', 'lat;lon']]
df = df.sort_values('income', ascending=False)
df

,inn,income,short_name,full_name,region,city,lat;lon
290837,4824095713,33821096000000,ООО &quot;АЛЬТЕРНАТИВА&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,48,Г. ЛИПЕЦК,52.600951;39.5658043
290836,4826133749,28276417000000,ООО &quot;ТЕТРАНИУМ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,48,Г. ЛИПЕЦК,52.6008681;39.5917492
290835,4825131001,11279905000000,ООО &quot;ТРЕЙДИНГ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,48,Г. ЛИПЕЦК,52.5379995;39.585263571415474
290834,7736050003,5179549285000,NaN,ГАЗПРОМ ОАО,15,ГОРОД ВЛАДИКАВКАЗ,43.024593;44.68211
290833,5504036333,2070008652000,ООО &quot;ГАЗПРОМНЕФТЬ-УРАЛ (НБ)&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,66,ГОРОД ЕКАТЕРИНБУРГ,56.8211021;60.6494182
...,...,...,...,...,...,...,...
3,5405477070,50002000,ООО &quot;АРСЕНАЛ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,54,ГОРОД НОВОСИБИРСК,55.0115196;82.9378134
5,5610234680,50002000,ООО &quot;АРК-ЦЕНТР&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,56,NaN,51.1985379;58.3070998
2,4717010585,50001000,ООО &quot;ГЕФЕС&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,47,Г. СЛАНЦЫ,59.1247154;28.0860501
1,3664006076,50001000,ГАУ &quot;УЧЕБНЫЙ АВТОКОМБИНАТ&quot;,ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ ВОРОНЕЖС...,36,Г. Павловск,50.4574176;40.1051697


In [47]:
downloaded = glob('data/*')
inns = list(map(lambda line : int(line[5:]), downloaded))

valid = list()
err = list()
for inn in tqdm(inns):
    path = f'data/{inn}/{inn}_report.xlsx'
    if os.path.isfile(path):
        ws = opx.load_workbook(path).active
        years = [ws.cell(row=8, column=ind).value for ind in range(3,20)]
        years = list(filter(None, years))
        # print(inn)
        
        maxYear = 0
        try:
            maxYear = int(years[-1])
            if int(years[-1]) >= 2021:
                valid.append(inn)
        except ValueError:
            if '2021' or '2022' in years[-1]:
                valid.append(inn)

100%|██████████| 7369/7369 [01:03<00:00, 116.44it/s]


In [56]:
df = df.loc[df['inn'].isin(valid)]

In [10]:
ll = df['lat;lon'].values
lat = []
lon = []
for el in tqdm(ll):
    temp = el.split(';')
    lat.append(float(temp[0]))
    lon.append(float(temp[1]))
    
df['latitude'] = lat
df['longitude'] = lon
df = df.drop(columns=['lat;lon'])
df

100%|██████████| 5013/5013 [00:00<00:00, 1160698.09it/s]


,Unnamed: 0,inn,income,short_name,full_name,region,city,latitude,longitude
0,290834,7736050003,5179549285000,NaN,ГАЗПРОМ ОАО,15,ГОРОД ВЛАДИКАВКАЗ,43.024593,44.682110
1,290833,5504036333,2070008652000,ООО &quot;ГАЗПРОМНЕФТЬ-УРАЛ (НБ)&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,66,ГОРОД ЕКАТЕРИНБУРГ,56.821102,60.649418
2,290832,5504036333,2070008652000,ООО &quot;ГАЗПРОМНЕФТЬ-УРАЛ (АЗС)&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,66,ГОРОД ЕКАТЕРИНБУРГ,56.821102,60.649418
3,290831,5504036333,2070008652000,ООО &quot;МАЛО-ЯМАЛЬСКОЕ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,89,ГОРОД НОВЫЙ УРЕНГОЙ,66.092257,76.686521
4,290829,8602060555,1524947700000,ПАО &quot;СУРГУТНЕФТЕГАЗ&quot;,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО &quot;СУРГУТНЕФ...,86,Г. СУРГУТ,61.250704,73.367743
...,...,...,...,...,...,...,...,...,...
5008,283276,6829045090,2413008000,ООО &quot;РАЙ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,68,Г. ТАМБОВ,52.724776,41.453203
5009,283273,4027131618,2411734000,ООО &quot;МАНУФАКТУРЫ БОСКО&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,40,Г. КАЛУГА,54.536984,36.284765
5010,283272,6452109727,2411636000,ООО &quot;НИТА-ФАРМ&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,64,Г. САРАТОВ,52.005230,47.810685
5011,283271,5204012980,2411567000,ООО &quot;ННПП-2&quot;,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ &quot...,52,NaN,55.783750,44.776319


In [ ]:
df.to_csv('company_df.csv')

In [17]:
df = pd.read_csv('company_df.csv')
df.reset_index(drop=True)
inn_list = df['inn']


debt = []
params_list = [[] for i in range(7)]
err = []
for inn in tqdm(inn_list):
    ws = opx.load_workbook(f'data/{inn}/{inn}_report.xlsx').active
    years = [ws.cell(row=8, column=ind).value for ind in range(3,15)]
    years = list(filter(None, years))

    LAST_COL = len(years) - 1
    
    try:
        debt.append(int(ws.cell(row=35, column=LAST_COL).value) + int(ws.cell(row=40, column=LAST_COL).value))
    except (ValueError, TypeError):
        debt.append(None)
        err.append(inn)
        
    for val_list, row_num in  zip(params_list, [24, 47, 49, 55, 56, 59, 61]):
        try:
            val_list.append(int(ws.cell(row=row_num, column=LAST_COL).value))
        except (ValueError, TypeError):
            val_list.append(None)
            
print(len(err))
                

100%|██████████| 5013/5013 [00:44<00:00, 112.87it/s]

18


In [19]:
df['Debt'] = debt

names = ['Assets', 'Revenue', 'Gross profit', 'Interest profit', 'Interest expences', 'EBIT', 'Profit']
for i in range(7):
    df[names[i]] = params_list[i]
    
df = pd.read_csv('company_fin_df.csv')
df['Interest rate'] = df['Interest expences'] / df['Debt']

df.to_csv('company_fin_df.csv')